In [1]:
require 'nn'
require 'rnn'

In [2]:
-- Some useful functions
function genNbyK(n, k, a, b)
    out = torch.LongTensor(n, k)
    for i=1, n do
        for j = 1, k do
            out[i][j] = torch.random(a, b)
        end
    end
    return out
end

function buildModel(model, vocabSize, embeddingSize, metric, adapt, use_cuda)
    -- Small experiments seem to show that the Tanh activations performed better\
    --      than the ReLU for the bow model
    if model == 'bow' then
        print(string.format("Running bag-of-words model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.Sum(2, 3, true)) -- Not averaging blows up model so keep this true
                    :add(nn.Tanh())
    else
        print(string.format("Running LSTM model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.SplitTable(2))
                    :add(nn.Sequencer(nn.LSTM(embeddingSize, embeddingSize)))
                    :add(nn.SelectTable(-1))            -- selects last state of the LSTM
                    :add(nn.Linear(embeddingSize, embeddingSize))
                    :add(nn.ReLU())
    end
    local queryLookup = sentenceLookup:clone("weight", "gradWeight") 
    local summaryLookup = sentenceLookup:clone("weight", "gradWeight")
    local pmodule = nn.ParallelTable()
                :add(sentenceLookup)
                :add(queryLookup)
                :add(summaryLookup)

    if model == 'bow' then
        nnmodel = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(nn.Tanh())
            :add(nn.Linear(embeddingSize * 3, 2))
    else
        nnmodel = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(nn.ReLU())
            :add(nn.Linear(embeddingSize * 3, 2))
    end

    if adapt then 
        print("Adaptive regularization")
        local logmod = nn.Sequential()
            :add(nn.Linear(embeddingSize * 3, 1))
            :add(nn.LogSigmoid())
            :add(nn.SoftMax())

        local regmod = nn.Sequential()
            :add(nn.Linear(embeddingSize * 3, 2))

        local fullmod = nn.ConcatTable()
            :add(regmod)
            :add(logmod)

        local final = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(fullmod)

        nnmodel = final
    end

    if use_cuda then
        return nnmodel:cuda()
    end
    return nnmodel
end

In [22]:
-- Setting parameters
n = 10
n_s = 5
k = 7
q = 5
a = 1
b = 100
embDim = 50
SELECT = 2
SKIP = 1

In [23]:
-- Simulating streams and queries
queries = genNbyK(n, q, a, b)

-- Note that the sentences are batched by sentence index so sentences[1] is the first sentence of each article
sentences = {}
for i=1, n_s do
    sentences[i] = genNbyK(n, k, a, b)
end

-- Using this to generate the optimal actions
true_actions = {}
for i=1, n_s do 
    ---- Simulating the data
    trueqValues = torch.rand(n, 2)                   
     ---- Generating the max values and getting the indices
    qMaxtrue, qindxtrue = torch.max(trueqValues, 2) 
    --- I want to select the qindx elements for each row
    true_actions[i] = torch.zeros(n, 2):scatter(2, qindxtrue, torch.ones(trueqValues:size()))
end

In [24]:
model = buildModel('bow', b, embDim, 'f1', false, false)

Running bag-of-words model to learn f1	


In [25]:
preds = model:forward({sentences[1], queries, torch.zeros(n, q)})
print(preds)

-0.0695  0.0120
-0.0304 -0.1922
 0.1785 -0.0060
-0.1531  0.0930
-0.2210  0.0182
 0.1283 -0.0919
 0.1178  0.1532
 0.2258 -0.0411
 0.0781  0.0294
 0.0017  0.0904
[torch.DoubleTensor of size 10x2]



In [26]:
qMax, qindx = torch.max(preds, 2)
actions = torch.zeros(n, 2):scatter(2, qindx, torch.ones(preds:size()))

In [27]:
actions:select(2, 1), qindx

 0
 1
 1
 0
 0
 1
 0
 1
 1
 0
[torch.DoubleTensor of size 10]

 2
 1
 1
 2
 2
 1
 2
 1
 1
 2
[torch.LongTensor of size 10x1]



In [31]:
torch.zeros(n, 15):scatter(2, qindx, sentences[1]:double())

  0  25   0   0   0   0   0   0   0   0   0   0   0   0   0
 62   0   0   0   0   0   0   0   0   0   0   0   0   0   0
 85   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0  70   0   0   0   0   0   0   0   0   0   0   0   0   0
  0  38   0   0   0   0   0   0   0   0   0   0   0   0   0
 58   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0  19   0   0   0   0   0   0   0   0   0   0   0   0   0
 14   0   0   0   0   0   0   0   0   0   0   0   0   0   0
 47   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0  57   0   0   0   0   0   0   0   0   0   0   0   0   0
[torch.DoubleTensor of size 10x15]



In [29]:
sentences[1]

 25   7  86  24  16  39  53
 62   2  22  59  70  19   3
 85  10  79  57  25  11  13
 70  22  85  51  97  76  94
 38  57   4  91  75  63   2
 58  27   6   6  50  91  65
 19  25  87  96  84  98  62
 14  74  33  29  38  24  80
 47  21  80  46  15  26  76
 57  60  75  48  31  21  97
[torch.LongTensor of size 10x7]



In [104]:
indices = torch.linspace(1, x:size(1), x:size(1) ):long()
selected = indices[x:eq(1)]

In [105]:
selected

  1
  2
  5
  6
  9
 10
[torch.LongTensor of size 6]



In [39]:
indx = torch.range(1, 10)
-- Summary index for selecting the words
sumindx = nn.MaskedSelect():forward({indx, actions:select(2,1):resize(10, 1):byte()})  --- sentences
-- need a LongTensor to do the selection
-- sumindx = torch.LongTensor(sumindx:size(1)):copy(sumindx)

In [ ]:
indx = torch.range(1, 10)
-- Summary index for selecting the words
sumindx = nn.MaskedSelect():forward({indx, actions:select(2,1):resize(10, 1):byte()})  --- sentences
-- need a LongTensor to do the selection
sumindx = torch.LongTensor(sumindx:size(1)):copy(sumindx)
--- This gives the wrong selection we want
print(sentences:index(1, sumindx))